Импорт библиотек.

In [1]:
from collections import Counter
from ultralytics import YOLO
import cv2
import os

Загрузка модели

In [2]:
model = YOLO("Dataset/yolov8s.pt")
model.fuse()

YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


Распознование объектов с видео и веб-камеры.

In [ ]:
cap = cv2.VideoCapture(0)
confidence_threshold = 0.4
while True:
    success, frame = cap.read()
    results = model(source=frame, show=True, conf = confidence_threshold)
    for data in results[0].boxes.data.tolist():
        cls = results[0].names[int(data[5])]
        xmn, ymn, xmx, ymx, conf = int(data[0]), int(data[1]), int(data[2]), int(data[3]), round(data[4],2)
        if conf < confidence_threshold:
            continue
        cv2.rectangle(frame, (xmn, ymn), (xmx, ymx), color=(0,0,255))
        cv2.putText(frame, f"{cls} {conf}", org=(xmn, ymn-20), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(0, 0, 255), thickness=2)
    cv2.imshow("Ручное", frame)
    if cv2.waitKey(2) == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
Подсчет точности для фото.

In [6]:
results = model('images/')
print(model.names)
k = 0
for result in results:
    prediction = result.boxes.cls.numpy()
    real = os.path.basename(result.path).split('_')[1:]
    real[-1] = real[-1].split('.')[0]
    real = list(map(int, real))
    intersection = Counter(real) & Counter(prediction)
    accuracy = int((sum(intersection.values())/max(len(real),len(prediction)) * 100))
    k+=accuracy
    print(f'{os.path.basename(result.path)} image prediction accuracy is {accuracy}%')
print(f"overall accuracy is {round(k/len(results),2)}%")


image 1/6 C:\Users\Acer\images\0_2_15.png: 352x640 1 car, 1 cat, 730.2ms
image 2/6 C:\Users\Acer\images\1_42_43_60.jpg: 448x640 1 fork, 1 knife, 1 dining table, 591.6ms
image 3/6 C:\Users\Acer\images\2_46_47_49.jpg: 640x640 1 banana, 1 apple, 1 orange, 717.7ms
image 4/6 C:\Users\Acer\images\3_0_15_16.jpeg: 448x640 1 person, 1 cat, 1 dog, 554.2ms
image 5/6 C:\Users\Acer\images\4_0_7_7.jpg: 384x640 3 persons, 2 trucks, 368.8ms
image 6/6 C:\Users\Acer\images\5_0_57_67.jpg: 448x640 1 person, 1 couch, 1 cell phone, 538.9ms
Speed: 5.8ms preprocess, 583.6ms inference, 4.2ms postprocess per image at shape (1, 3, 448, 640)
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag